This code will train one model at a time on however many datasets you wish.
HP are randomly selected from a grid of appropriate hp's for this task.


In [4]:
### -------- Part 0: CONFIGURATION -------- ###
import os, sys, pickle, random, time, itertools
from datetime import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from Utils.utils import segment_timeseries_v2, TimeseriesDataset, MinMaxScale_datasets
from Utils.prediction_models import TemporalCNN_Pr, SimpleFFNN_Pr, SimpleLSTM_Pr
from Utils.prediction_utils import train_valid_model_fast, test_trained_model

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# --- User-configurable options
farm = 'Kelmarsh'  # Can be changed
model_choice = 'cnn'  # 'cnn', 'lstm', or 'ffnn'
random_seed = 42
random_select = 22
target_feature = 'Generator bearing rear temperature (°C)'
sim = 1  # Simulation run number

# --- Paths and loading
with open(f'./1_healthy_datasets/{farm}_HealthyDatasets.pkl', 'rb') as f:
    healthy_datasets = pickle.load(f)
with open(f'./2_ordered_inputs/{farm}_ordered_inputs.pkl', 'rb') as f:
    chosen_features_dict = pickle.load(f)

# --- Normalization
healthy_datasets = MinMaxScale_datasets(healthy_datasets, target_feature)

# --- Configuration lists
dataset_names = [list(healthy_datasets.keys())[1]]  # Or more
fs_methods = ['LC', 'DT', 'MI', 'MLPSHAP']
pred_models = {
    'cnn': TemporalCNN_Pr,
    'lstm': SimpleLSTM_Pr,
    'ffnn': SimpleFFNN_Pr
}

### -------- Part 1: HYPERPARAMETER GRID -------- ###
fixed_hyperparams = {'steps_ahead': 0}

hp_grid = {
    'cnn': {
        'num_features': [10, 20, 30],
        'window_size': [9, 18],
        'batch_size': [32, 64],
        'num_epochs': [200],
        'num_channels': [32, 64],
        'num_layers': [1, 2],
        'kernel_size': [3, 5],
        'learning_rate': [0.001, 0.0001]
    },
    'ffnn': {
        'num_features': [10, 20, 30],
        'window_size': [9, 18],
        'batch_size': [32, 64],
        'num_epochs': [200],
        'hidden_size': [64, 128, 192],
        'num_layers': [1, 2, 3],
        'learning_rate': [0.001, 0.0001]
    },
    'lstm': {
        'num_features': [10, 20, 30],
        'window_size': [9, 18],
        'batch_size': [32, 64],
        'num_epochs': [200],
        'hidden_size': [32, 64, 128],
        'num_layers_lstm': [1, 2, 3],
        'learning_rate': [0.001, 0.0001]
    }
}

# --- Set Random seed for HP combinations
random.seed(random_seed)
param_names = list(hp_grid[model_choice].keys())
hp_combinations = list(itertools.product(*hp_grid[model_choice].values()))
random_combinations = random.sample(hp_combinations, random_select)

# save hp's
results_dir = f'./3_results'
os.makedirs(results_dir, exist_ok=True)
df_random_combinations = pd.DataFrame(random_combinations, columns=param_names)
df_random_combinations.to_csv(f'{results_dir}/HP_combinations_sim{sim}_{model_choice}.csv', index=False)

### -------- Part 2: TRAINING LOOP -------- ###
trained_models = {}
metrics = {}
selected_datasets = {}

print(f'Running training for Farm: {farm}, Model: {model_choice}, sim: {sim}')

for dataset_name in dataset_names:
    train_set = healthy_datasets[dataset_name]['train']
    valid_set = healthy_datasets[dataset_name]['valid']
    test_set = healthy_datasets[dataset_name]['test']

    for fs_method in fs_methods:
        print(f'\n-- FS Method: {fs_method} --')
        hp_set_counter = 0
        best_loss_for_HP_set = float('inf')

        all_features = chosen_features_dict[f'{dataset_name}_{fs_method}']

        for combination in random_combinations:
            
            #zip hps
            hp_dict = dict(zip(param_names, combination))

            # Data preparation
            chosen_features = all_features[:hp_dict['num_features']]
            X_train, y_train = segment_timeseries_v2(train_set[chosen_features].values, train_set[target_feature].values, hp_dict['window_size'], fixed_hyperparams['steps_ahead'])
            X_valid, y_valid = segment_timeseries_v2(valid_set[chosen_features].values, valid_set[target_feature].values, hp_dict['window_size'], fixed_hyperparams['steps_ahead'])
            X_test, y_test = segment_timeseries_v2(test_set[chosen_features].values, test_set[target_feature].values, hp_dict['window_size'], fixed_hyperparams['steps_ahead'])

            train_loader = DataLoader(TimeseriesDataset(X_train, y_train), batch_size=hp_dict['batch_size'], shuffle=False)
            valid_loader = DataLoader(TimeseriesDataset(X_valid, y_valid), batch_size=hp_dict['batch_size'], shuffle=False)
            test_loader = DataLoader(TimeseriesDataset(X_test, y_test), batch_size=hp_dict['batch_size'], shuffle=False)

            # Model definition
            if model_choice == 'cnn':
                                    model = pred_models[model_choice](
                                        input_size=hp_dict['num_features'],
                                        num_channels=hp_dict['num_channels'],
                                        num_layers=hp_dict['num_layers'],
                                        kernel_size=hp_dict['kernel_size']
                                    )

            elif model_choice == 'lstm':
                                    model = pred_models[model_choice](
                                        input_size=hp_dict['num_features'],
                                        hidden_size=hp_dict['hidden_size'],
                                        num_layers_lstm=hp_dict['num_layers_lstm']
                                    )

            elif model_choice == 'ffnn':
                                    model = pred_models[model_choice](
                                        input_size=hp_dict['num_features'] * hp_dict['window_size'],
                                        hidden_size=hp_dict['hidden_size'],
                                        num_layers=hp_dict['num_layers']
                                    )
            # Training
            print(f'Model has {count_parameters(model):,} trainable parameters')
            criterion = nn.MSELoss(reduction='sum')
            optimizer = optim.Adam(model.parameters(), lr=hp_dict['learning_rate'])
            best_model_state, best_valid_loss = train_valid_model_fast(model, criterion, optimizer, train_loader, valid_loader, hp_dict['num_epochs'])

            # Save
            key = f'{dataset_name}_{model_choice}_{fs_method}_sim{sim}'
            trained_models[key] = best_model_state

            # Test
            model.load_state_dict(best_model_state)
            metrics[key] = {
                'test': test_trained_model(model, test_loader),
                'valid': test_trained_model(model, valid_loader)
            }

            # Save artifacts
            model_dir = f'./3_results/models/sim{sim}_{model_choice}/{dataset_name}'
            metrics_dir = f'./3_results/metrics/sim{sim}_{model_choice}/{dataset_name}'
            os.makedirs(model_dir, exist_ok=True)
            os.makedirs(metrics_dir, exist_ok=True)

            with open(f'{model_dir}/Model_{farm}_{dataset_name}_{model_choice}_{fs_method}_hpset{hp_set_counter}.pkl', 'wb') as f:
                pickle.dump(best_model_state, f)

            with open(f'{metrics_dir}/Metrics_{farm}_{dataset_name}_{model_choice}_{fs_method}_hpset{hp_set_counter}.pkl', 'wb') as f:
                pickle.dump(metrics[key], f)

            print(f'[Set {hp_set_counter+1}/{random_select}] Loss: {best_valid_loss:.4f}')
            hp_set_counter += 1

            if best_valid_loss < best_loss_for_HP_set:
                best_loss_for_HP_set = best_valid_loss

        print(f'Best loss for {fs_method}_{dataset_name}: {best_loss_for_HP_set:.4f}')


Running training for Farm: Kelmarsh, Model: cnn, sim: 1

-- FS Method: LC --
Model has 4,898 trainable parameters
Epoch 2 took 2.1722 seconds
Epoch 2/200: best validation loss so far: 13.1466


KeyboardInterrupt: 